In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyspark as ps
import scipy.stats as stats
from datetime import datetime as dt
import sys
sys.path.append("..")
from src.support_functions import fixing_datetime

plt.style.use('ggplot')

In [2]:
turnstile2019_df = pd.read_csv('data/Turnstile_Usage_Data__2019.csv', low_memory=False)
turnstile2020_df = pd.read_csv('data/Turnstile_Usage_Data__2020.csv', low_memory=False)

## MTA Ridership 2019

In [3]:
turnstile2019_df.columns = [column.strip() for column in turnstile2019_df.columns]

In [4]:
turnstile2019_df.columns

Index(['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date',
       'Time', 'Description', 'Entries', 'Exits'],
      dtype='object')

In [5]:
turnstile2019_df.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A033,R170,02-06-00,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,769115,559221
1,A033,R170,02-00-04,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,6483080,4945335
2,A033,R170,02-00-03,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,7191422,8417203
3,A033,R170,02-00-02,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,14983900,14554087
4,A033,R170,02-06-01,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,71047673,20925389


In [6]:
turnstile2019_df.Date.value_counts().sort_index()

01/01/2019    28742
01/02/2019    28691
01/03/2019    28584
01/04/2019    29388
01/05/2019    28619
              ...  
12/26/2019    29466
12/27/2019    29423
12/29/2018    28591
12/30/2018    28648
12/31/2018    28959
Name: Date, Length: 343, dtype: int64

In [7]:
turnstile2019_df = fixing_datetime(turnstile2019_df)

In [8]:
turnstile2019_df.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
2799675,A002,R051,02-03-05,59 ST,NQR456W,BMT,2019-09-30,00:00:00,REGULAR,11658268,1416355
2799676,A002,R051,02-03-01,59 ST,NQR456W,BMT,2019-09-30,00:00:00,REGULAR,1270710,1935481
2799677,A002,R051,02-03-02,59 ST,NQR456W,BMT,2019-09-30,00:00:00,REGULAR,6357257,8509403
2799678,A002,R051,02-03-06,59 ST,NQR456W,BMT,2019-09-30,00:00:00,REGULAR,9125177,502764
2799679,A002,R051,02-05-00,59 ST,NQR456W,BMT,2019-09-30,00:00:00,REGULAR,168,0


In [9]:
(turnstile2019_df
 .groupby(["C/A", "Unit", "SCP", "Station", "Date", "Time"])  
 .Entries.count()
 .reset_index()  # or use as_index = False; otherwise makes groupby columns new index 
 .sort_values("Entries", ascending=False)).head(5)

,C/A,Unit,SCP,Station,Date,Time,Entries
3623696,R101,R001,02-00-04,SOUTH FERRY,2019-09-17,05:00:00,4
3627436,R101,R001,02-00-07,SOUTH FERRY,2019-09-17,05:00:00,4
3394843,PTH07,R550,00-01-04,CITY / BUS,2019-09-20,07:49:35,2
4930091,R317,R408,01-05-00,SIMPSON ST,2019-09-15,21:00:00,2
4930103,R317,R408,01-05-00,SIMPSON ST,2019-09-17,21:00:00,2


In [10]:
turnstile2019_df.Description.value_counts()

REGULAR       6018246
RECOVR AUD      25722
Name: Description, dtype: int64

In [11]:
# Get rid of the duplicate entry
turnstile2019_df.sort_values(["C/A", "Unit", "SCP", "Station", "Date", "Time"], inplace=True, ascending=False)
turnstile2019_df.drop_duplicates(subset=["C/A", "Unit", "SCP", "Station", "Date", "Time"], inplace=True)

In [12]:
(turnstile2019_df
 .groupby(["C/A", "Unit", "SCP", "Station", "Date", "Time"])
 .Entries.count()
 .reset_index()
 .sort_values("Entries", ascending=False)).head(5)

,C/A,Unit,SCP,Station,Date,Time,Entries
0,A002,R051,02-00-00,59 ST,2019-03-02,03:00:00,1
3892599,R139,R031,04-06-00,34 ST-PENN STA,2019-05-07,16:00:00,1
3892619,R139,R031,04-06-00,34 ST-PENN STA,2019-05-11,00:00:00,1
3892618,R139,R031,04-06-00,34 ST-PENN STA,2019-05-10,20:00:00,1
3892617,R139,R031,04-06-00,34 ST-PENN STA,2019-05-10,16:00:00,1


In [13]:
# Drop Exits and Desc columns. To prevent errors in multiple run of cell, 
# errors on drop is ignored (e.g. if some columns were dropped already)
turnstile2019_df = turnstile2019_df.drop(["Exits", "Description"], axis=1, errors="ignore")

In [14]:
turnstile2019_daily = turnstile2019_df.groupby(["C/A", "Unit", "SCP", "Station", "Date"]).Entries.first().reset_index()

In [15]:
turnstile2019_daily.head()

,C/A,Unit,SCP,Station,Date,Entries
0,A002,R051,02-00-00,59 ST,2019-03-02,6964320
1,A002,R051,02-00-00,59 ST,2019-03-03,6964888
2,A002,R051,02-00-00,59 ST,2019-03-04,6966123
3,A002,R051,02-00-00,59 ST,2019-03-05,6967602
4,A002,R051,02-00-00,59 ST,2019-03-06,6969056


In [16]:
turnstile2019_daily[["Previous_Date", "Previous_Entries"]] = (turnstile2019_daily
                                                       .groupby(["C/A", "Unit", "SCP", "Station"])["Date", "Entries"]
                                                       .transform(lambda grp: grp.shift(1)))
# transform() takes a function as parameter
# shift moves the index by the number of periods given (positive or negative)

/Users/thomasjoy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [17]:
turnstile2019_daily.head()

,C/A,Unit,SCP,Station,Date,Entries,Previous_Date,Previous_Entries
0,A002,R051,02-00-00,59 ST,2019-03-02,6964320,NaT,NaN
1,A002,R051,02-00-00,59 ST,2019-03-03,6964888,2019-03-02,6964320.0
2,A002,R051,02-00-00,59 ST,2019-03-04,6966123,2019-03-03,6964888.0
3,A002,R051,02-00-00,59 ST,2019-03-05,6967602,2019-03-04,6966123.0
4,A002,R051,02-00-00,59 ST,2019-03-06,6969056,2019-03-05,6967602.0


In [18]:
# Drop the rows for first date
turnstile2019_daily.dropna(subset=["Previous_Date"], axis=0, inplace=True)
# axis = 0 means index (=1 means column)

In [19]:
turnstile2019_daily[turnstile2019_daily["Entries"] < turnstile2019_daily["Previous_Entries"]].head()

,C/A,Unit,SCP,Station,Date,Entries,Previous_Date,Previous_Entries
2311,A002,R051,02-06-00,59 ST,2019-07-13,524,2019-07-12,258249.0
4318,A007,R079,01-05-01,5 AV/59 ST,2019-08-10,0,2019-08-02,1.0
7728,A011,R080,01-03-00,57 ST-7 AV,2019-08-20,885829427,2019-08-19,885829659.0
7729,A011,R080,01-03-00,57 ST-7 AV,2019-08-21,885828919,2019-08-20,885829427.0
7730,A011,R080,01-03-00,57 ST-7 AV,2019-08-22,885828367,2019-08-21,885828919.0


In [20]:
(turnstile2019_daily[turnstile2019_daily["Entries"] < turnstile2019_daily["Previous_Entries"]].groupby(["C/A", "Unit", "SCP", "Station"]).size())

C/A    Unit  SCP       Station        
A002   R051  02-06-00  59 ST                1
A007   R079  01-05-01  5 AV/59 ST           1
A011   R080  01-03-00  57 ST-7 AV          42
A015   R081  00-03-01  49 ST                1
A025   R023  01-03-01  34 ST-HERALD SQ    198
                                         ... 
R645   R110  00-03-00  FLATBUSH AV-B.C      3
             00-03-02  FLATBUSH AV-B.C      2
R646   R110  01-00-01  FLATBUSH AV-B.C    198
R730   R431  00-00-04  EASTCHSTER/DYRE    191
TRAM1  R468  00-00-01  RIT-MANHATTAN        1
Length: 745, dtype: int64

In [21]:
def get_daily_counts(row, max_counter):
    counter = row["Entries"] - row["Previous_Entries"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        return 0
    return counter

# If counter is > 1 million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
turnstile2019_daily["Daily_Entries"] = turnstile2019_daily.apply(get_daily_counts, axis=1, max_counter=1000000) 


In [22]:
turnstile2019_daily.head()

,C/A,Unit,SCP,Station,Date,Entries,Previous_Date,Previous_Entries,Daily_Entries
1,A002,R051,02-00-00,59 ST,2019-03-03,6964888,2019-03-02,6964320.0,568.0
2,A002,R051,02-00-00,59 ST,2019-03-04,6966123,2019-03-03,6964888.0,1235.0
3,A002,R051,02-00-00,59 ST,2019-03-05,6967602,2019-03-04,6966123.0,1479.0
4,A002,R051,02-00-00,59 ST,2019-03-06,6969056,2019-03-05,6967602.0,1454.0
5,A002,R051,02-00-00,59 ST,2019-03-07,6970476,2019-03-06,6969056.0,1420.0


In [23]:
station_ridership_2019 = turnstile2019_daily.groupby(['Station'])['Daily_Entries'].sum().sort_values(ascending = False).reset_index()

In [24]:
station_ridership_2019.head()

,Station,Daily_Entries
0,34 ST-PENN STA,33482904.0
1,GRD CNTRL-42 ST,30459852.0
2,34 ST-HERALD SQ,23597515.0
3,23 ST,23182538.0
4,14 ST-UNION SQ,20071036.0


## MTA Ridership 2020

In [25]:
turnstile2020_df.columns = [column.strip() for column in turnstile2020_df.columns]
turnstile2020_df.columns

Index(['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date',
       'Time', 'Description', 'Entries', 'Exits'],
      dtype='object')

In [26]:
turnstile2020_df = fixing_datetime(turnstile2020_df)

In [27]:
(turnstile2020_df
 .groupby(["C/A", "Unit", "SCP", "Station", "Date", "Time"])  
 .Entries.count()
 .reset_index()  # or use as_index = False; otherwise makes groupby columns new index 
 .sort_values("Entries", ascending=False)).head(5)

,C/A,Unit,SCP,Station,Date,Time,Entries
317111,A060,R001,00-00-00,WHITEHALL S-FRY,2020-08-01,13:00:00,4
956551,H009,R235,00-03-00,BEDFORD AV,2020-03-22,12:00:00,4
2513363,N333A,R141,00-00-00,FOREST HILLS 71,2020-08-26,05:00:00,4
1158815,J023,R436,00-00-00,NORWOOD AV,2020-08-25,05:00:00,4
338220,A064,R311,00-03-01,BOWERY,2020-08-17,16:00:00,4


In [28]:
turnstile2020_df.Description.value_counts()

REGULAR       8250524
RECOVR AUD      30658
Name: Description, dtype: int64

In [29]:
turnstile2020_df.sort_values(["C/A", "Unit", "SCP", "Station", "Date", "Time"], inplace=True, ascending=False)
turnstile2020_df.drop_duplicates(subset=["C/A", "Unit", "SCP", "Station", "Date", "Time"], inplace=True)

In [30]:
(turnstile2019_df
 .groupby(["C/A", "Unit", "SCP", "Station", "Date", "Time"])
 .Entries.count()
 .reset_index()
 .sort_values("Entries", ascending=False)).head(5)

,C/A,Unit,SCP,Station,Date,Time,Entries
0,A002,R051,02-00-00,59 ST,2019-03-02,03:00:00,1
3892599,R139,R031,04-06-00,34 ST-PENN STA,2019-05-07,16:00:00,1
3892619,R139,R031,04-06-00,34 ST-PENN STA,2019-05-11,00:00:00,1
3892618,R139,R031,04-06-00,34 ST-PENN STA,2019-05-10,20:00:00,1
3892617,R139,R031,04-06-00,34 ST-PENN STA,2019-05-10,16:00:00,1


In [31]:
turnstile2020_df = turnstile2020_df.drop(["Exits", "Description"], axis=1, errors="ignore")

In [32]:
turnstile2020_daily = turnstile2020_df.groupby(["C/A", "Unit", "SCP", "Station", "Date"]).Entries.first().reset_index()

In [33]:
turnstile2020_daily.head()

,C/A,Unit,SCP,Station,Date,Entries
0,A002,R051,02-00-00,59 ST,2020-03-01,7396077
1,A002,R051,02-00-00,59 ST,2020-03-02,7397350
2,A002,R051,02-00-00,59 ST,2020-03-03,7398638
3,A002,R051,02-00-00,59 ST,2020-03-04,7399918
4,A002,R051,02-00-00,59 ST,2020-03-05,7401196


In [34]:
turnstile2020_daily[["Previous_Date", "Previous_Entries"]] = (turnstile2019_daily
                                                       .groupby(["C/A", "Unit", "SCP", "Station"])["Date", "Entries"]
                                                       .transform(lambda grp: grp.shift(1)))

/Users/thomasjoy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [35]:
turnstile2020_daily.head()

,C/A,Unit,SCP,Station,Date,Entries,Previous_Date,Previous_Entries
0,A002,R051,02-00-00,59 ST,2020-03-01,7396077,NaT,NaN
1,A002,R051,02-00-00,59 ST,2020-03-02,7397350,NaT,NaN
2,A002,R051,02-00-00,59 ST,2020-03-03,7398638,2019-03-03,6964888.0
3,A002,R051,02-00-00,59 ST,2020-03-04,7399918,2019-03-04,6966123.0
4,A002,R051,02-00-00,59 ST,2020-03-05,7401196,2019-03-05,6967602.0


In [36]:
turnstile2020_daily.dropna(subset=["Previous_Date"], axis=0, inplace=True)

In [37]:
turnstile2020_daily[turnstile2020_daily["Entries"] < turnstile2020_daily["Previous_Entries"]].head()

,C/A,Unit,SCP,Station,Date,Entries,Previous_Date,Previous_Entries
798,A002,R051,02-03-01,59 ST,2020-09-20,1486556,2019-03-03,6102678.0
799,A002,R051,02-03-01,59 ST,2020-09-21,1487071,2019-03-04,6103713.0
800,A002,R051,02-03-01,59 ST,2020-09-22,1487602,2019-03-05,6104982.0
801,A002,R051,02-03-01,59 ST,2020-09-23,1488150,2019-03-06,6106310.0
802,A002,R051,02-03-01,59 ST,2020-09-24,1488708,2019-03-07,6107539.0


In [38]:
(turnstile2020_daily[turnstile2020_daily["Entries"] < turnstile2020_daily["Previous_Entries"]].groupby(["C/A", "Unit", "SCP", "Station"]).size())

C/A   Unit  SCP       Station        
A002  R051  02-03-01  59 ST                6
            02-03-03  59 ST               10
            02-03-04  59 ST               12
            02-05-01  59 ST               24
            02-06-00  59 ST               84
                                        ... 
R623  R061  00-00-02  NOSTRAND AV        200
R625  R062  01-00-00  CROWN HTS-UTICA    200
R626  R062  00-05-00  CROWN HTS-UTICA    147
            00-05-01  CROWN HTS-UTICA    200
R627  R063  00-00-00  SUTTER AV-RUTLD     36
Length: 3232, dtype: int64

In [39]:
turnstile2020_daily["Daily_Entries"] = turnstile2020_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

In [50]:
turnstile2020_daily.head(20)

,C/A,Unit,SCP,Station,Date,Entries,Previous_Date,Previous_Entries,Daily_Entries
2,A002,R051,02-00-00,59 ST,2020-03-03,7398638,2019-03-03,6964888.0,433750.0
3,A002,R051,02-00-00,59 ST,2020-03-04,7399918,2019-03-04,6966123.0,433795.0
4,A002,R051,02-00-00,59 ST,2020-03-05,7401196,2019-03-05,6967602.0,433594.0
5,A002,R051,02-00-00,59 ST,2020-03-06,7402441,2019-03-06,6969056.0,433385.0
6,A002,R051,02-00-00,59 ST,2020-03-07,7403197,2019-03-07,6970476.0,432721.0
7,A002,R051,02-00-00,59 ST,2020-03-08,7403667,2019-03-08,6971959.0,431708.0
8,A002,R051,02-00-00,59 ST,2020-03-09,7404827,2019-03-09,6972662.0,432165.0
9,A002,R051,02-00-00,59 ST,2020-03-10,7405970,2019-03-10,6973098.0,432872.0
10,A002,R051,02-00-00,59 ST,2020-03-11,7407050,2019-03-11,6974378.0,432672.0
11,A002,R051,02-00-00,59 ST,2020-03-12,7408111,2019-03-12,6975879.0,432232.0


In [40]:
station_ridership_2020 = turnstile2020_daily.groupby(['Station'])['Daily_Entries'].sum().sort_values(ascending = False).reset_index()

In [41]:
station_ridership_2020

,Station,Daily_Entries
0,FULTON ST,2.076856e+09
1,34 ST-PENN STA,1.374850e+09
2,23 ST,1.332654e+09
3,CORTLANDT ST,1.324231e+09
4,WTC-CORTLANDT,1.304341e+09


In [42]:
station_ridership_2019.head()

,Station,Daily_Entries
0,34 ST-PENN STA,33482904.0
1,GRD CNTRL-42 ST,30459852.0
2,34 ST-HERALD SQ,23597515.0
3,23 ST,23182538.0
4,14 ST-UNION SQ,20071036.0


In [43]:
turnstile2020_daily.head()

,C/A,Unit,SCP,Station,Date,Entries,Previous_Date,Previous_Entries,Daily_Entries
2,A002,R051,02-00-00,59 ST,2020-03-03,7398638,2019-03-03,6964888.0,433750.0
3,A002,R051,02-00-00,59 ST,2020-03-04,7399918,2019-03-04,6966123.0,433795.0
4,A002,R051,02-00-00,59 ST,2020-03-05,7401196,2019-03-05,6967602.0,433594.0
5,A002,R051,02-00-00,59 ST,2020-03-06,7402441,2019-03-06,6969056.0,433385.0
6,A002,R051,02-00-00,59 ST,2020-03-07,7403197,2019-03-07,6970476.0,432721.0


In [44]:
daily_ridership_2020 = turnstile2020_daily.groupby(['Date'])['Daily_Entries'].sum().sort_values(ascending = False).reset_index()

In [45]:
turnstile2019_daily.head()

,C/A,Unit,SCP,Station,Date,Entries,Previous_Date,Previous_Entries,Daily_Entries
1,A002,R051,02-00-00,59 ST,2019-03-03,6964888,2019-03-02,6964320.0,568.0
2,A002,R051,02-00-00,59 ST,2019-03-04,6966123,2019-03-03,6964888.0,1235.0
3,A002,R051,02-00-00,59 ST,2019-03-05,6967602,2019-03-04,6966123.0,1479.0
4,A002,R051,02-00-00,59 ST,2019-03-06,6969056,2019-03-05,6967602.0,1454.0
5,A002,R051,02-00-00,59 ST,2019-03-07,6970476,2019-03-06,6969056.0,1420.0


In [46]:
daily_ridership_2019 = turnstile2019_daily.groupby(['Date'])['Daily_Entries'].sum().sort_values(ascending = False).reset_index()

In [48]:
daily_ridership_2019.head()

,Date,Daily_Entries
0,2019-06-22,39689258.0
1,2019-08-10,37843748.0
2,2019-06-11,16775539.0
3,2019-06-10,15552156.0
4,2019-06-05,11927078.0


In [47]:
daily_ridership_2020.head()

,Date,Daily_Entries
0,2020-09-23,444924991.0
1,2020-09-24,444792361.0
2,2020-09-25,444590845.0
3,2020-09-05,444015481.0
4,2020-09-07,443992903.0
